In [14]:
import pandas as pd
import requests
import math
from decimal import Decimal
import json

In [15]:
symbol = "SOLUSDT"
url = "https://api.binance.com/api/v1/depth"

interval = Decimal('0.1')

params = {
    "symbol": symbol,
    "limit": 100,
}

In [20]:
data = requests.get(url, params).json()
print(data)

{'lastUpdateId': 25122842850, 'bids': [['199.63000000', '82.30700000'], ['199.62000000', '35.91700000'], ['199.61000000', '53.78000000'], ['199.60000000', '207.39700000'], ['199.59000000', '48.99400000'], ['199.58000000', '31.42400000'], ['199.57000000', '65.81500000'], ['199.56000000', '82.49700000'], ['199.55000000', '66.15500000'], ['199.54000000', '120.95600000'], ['199.53000000', '204.57700000'], ['199.52000000', '92.56400000'], ['199.51000000', '961.86000000'], ['199.50000000', '97.89300000'], ['199.49000000', '225.20200000'], ['199.48000000', '96.22200000'], ['199.47000000', '462.56400000'], ['199.46000000', '2.21400000'], ['199.45000000', '83.85900000'], ['199.44000000', '158.44300000'], ['199.43000000', '279.19200000'], ['199.42000000', '513.40000000'], ['199.41000000', '419.97200000'], ['199.40000000', '10.74200000'], ['199.39000000', '19.79400000'], ['199.38000000', '87.94100000'], ['199.37000000', '607.67000000'], ['199.36000000', '10.80400000'], ['199.35000000', '80.129000